In [ ]:
pip install djitellopy opencv-python numpy

In [ ]:
from djitellopy import Tello
import cv2
import numpy as np
import time

#####################################
# Initialization of coordinates
#####################################
# x points straight, y points left, z points up
# Initial starting position of ego drone (at the center of the picnic blanket)
ego_x = 0
x_array = []
ego_y = 0
y_array = []
ego_z = 0
z_array = []

# Define bounding box around initial starting point (200 x 200)
# Box extends 100 units in each direction from center
BOX_SIZE = 200 # random estimate rn for the bounding box of the ODD, will test based on picnic blanket
BOX_HALF = BOX_SIZE / 2  # 100 units
box_x_min = ego_x - BOX_HALF
box_x_max = ego_x + BOX_HALF
box_y_min = ego_y - BOX_HALF
box_y_max = ego_y + BOX_HALF

# Adversary drone starting position (relative to ego drone start)
# You can adjust these offsets to position the adversary drone
adversary_offset_x = 150  # 150 units forward from ego start
adversary_offset_y = 100  # 100 units left from ego start
adversary_drone_x = ego_x + adversary_offset_x
adversary_drone_y = ego_y + adversary_offset_y

#####################################
# Initialization of motion params
#####################################
upward_initialization = 30
left_movement  = 50
right_movement = 50
forward_movement = 50
backward_movement = 50

#####################################
# Initialization of drone
#####################################
tello = Tello()
print("Connecting to Tello...")
tello.connect()
print(f"Battery level: {tello.get_battery()}%")
tello.streamon()

print("Press 't' to takeoff, 'q' to land and quit.")
drone_in_air = False


#####################################
# Drone maneuverance after takeoff
#####################################

try:
    while True:
        frame = tello.get_frame_read().frame
        frame = cv2.resize(frame, (720, 480))
        h, w, _ = frame.shape

        # Display the frame
        cv2.imshow("Tello Tracking Feed", frame)

        key = cv2.waitKey(1) & 0xFF

        # Controls for drone
        if key == ord('t') and not drone_in_air:
            tello.takeoff()
            drone_in_air = True
            print("Drone is now airborne and hovering.")
            tello.move_up(upward_initialization)  # move up 30 cm after takeoff
            ego_z += upward_initialization
            z_array.append(upward_initialization)
            print("Drone moved up 30 cm.")
        
        if drone_in_air:
            # WASD key controls for movement
            if key == ord('w'):  # W key - Move forward
                print("Moving forward.")
                ego_x += forward_movement
                x_array.append(forward_movement)
                tello.move_forward(forward_movement)  # Increase movement distance for testing
                # Calculate distance between ego and adversary
                dx = ego_x - adversary_drone_x
                dy = ego_y - adversary_drone_y
                print(f"dx (ego_x - adversary_x): {dx}, dy (ego_y - adversary_y): {dy}")
                
            elif key == ord('a'):  # A key - Move left
                print("Moving left.")
                ego_y += left_movement
                y_array.append(left_movement)
                tello.move_left(left_movement)  # Increase movement distance for testing
                # Calculate distance between ego and adversary
                dx = ego_x - adversary_drone_x
                dy = ego_y - adversary_drone_y
                print(f"dx (ego_x - adversary_x): {dx}, dy (ego_y - adversary_y): {dy}")
                
            elif key == ord('s'):  # S key - Move backward
                print("Moving backward.")
                ego_x -= backward_movement
                x_array.append(-backward_movement)
                tello.move_back(backward_movement)  # Increase movement distance for testing
                # Calculate distance between ego and adversary
                dx = ego_x - adversary_drone_x
                dy = ego_y - adversary_drone_y
                print(f"dx (ego_x - adversary_x): {dx}, dy (ego_y - adversary_y): {dy}")
                
            elif key == ord('d'):  # D key - Move right
                print("Moving right.")
                ego_y -= right_movement
                y_array.append(-right_movement)
                tello.move_right(right_movement)  # Increase movement distance for testing
                # Calculate distance between ego and adversary
                dx = ego_x - adversary_drone_x
                dy = ego_y - adversary_drone_y
                print(f"dx (ego_x - adversary_x): {dx}, dy (ego_y - adversary_y): {dy}")
                
        # Landing control
        if key == ord('q'):
            if drone_in_air:
                tello.land()
                print("Drone has landed.")
            break


#####################################
# Landing the drone / finish program
#####################################
except Exception as e:
    print(f"Error occurred: {e}")
    if drone_in_air:
        print("Error occurred. Landing the drone.")
        tello.land()

except KeyboardInterrupt:
    print("Keyboard Interrupt received. Landing the drone.")
    if drone_in_air:
        tello.land()
    tello.streamoff()
    cv2.destroyAllWindows()

finally:
    tello.streamoff()
    cv2.destroyAllWindows()
